# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

# Import

In [ ]:
from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.preprocessing.preprocessor import *
from debruiteur.noise.noise import *
from debruiteur.utils.utils import plot_im_grid
from debruiteur.metrics.metrics import *
import cv2
from skimage import measure

# Configurations

In [ ]:
noise_class_list = [
    GaussianNoise(mean=0, std=10),
    PoissonNoise(),
    UniformNoise(amplitude=100),
    SaltPepperNoise(p=0.3),
    SquareMaskNoise(mask_shape=(10, 10), freq=0.1),
    SpeckleNoise(),
    AveragingBlurNoise(),
    GaussianBlurNoise(),
    MedianBlurNoise()
]

# Create Dataframe

In [ ]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, 'images'))
df_resized = make_resized_dataframe(df_original, os.path.join(working_dir, 'resized_images'))
df_noised = make_noised_dataframe(df_resized, noise_clas_list, os.path.join(working_dir, 'noised_images'))